In [ ]:
!pip install rasterio
!apt install imagemagick

In [ ]:
import json
import requests
import os
import cv2
import urllib
import numpy as np
import matplotlib.pyplot as plt
import sys
from subprocess import call
import rasterio
from osgeo import gdal
from os import path
from rasterio.mask import mask
from collections import Counter
from tensorflow.keras.models import load_model

In [ ]:
# API Key used to access the API
os.environ['PL_API_KEY']='5affbba496b5487b8ca1ba4e89a2cd8d'
PLANET_API_KEY = os.getenv('PL_API_KEY')

In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Setup Planet API
URL = "https://api.planet.com/data/v1"
session = requests.Session()
session.auth = (PLANET_API_KEY, "")

In [ ]:
stats_url = "{}/stats".format(URL)
print(stats_url)

https://api.planet.com/data/v1/stats


In [ ]:
# Iterate through images to find the highest and lowest latidude and longitude coordinates
def get_corners(features): 
  longs = []
  lats = []
  for f in features:
    coords = f["geometry"]["coordinates"][0]
    longs.extend([coords[0][0], coords[1][0], coords[2][0], coords[3][0]])
    lats.extend([coords[0][1], coords[1][1], coords[2][1], coords[3][1]])

  return max(lats), min(lats), max(longs), min(longs)
  

In [ ]:
# Save search results returned by Planet
def save_pngs(features, path, search_id, res):
  png_imgs = []
  for feature in features:
    id = feature["id"]
    thumb_url = feature["_links"]["thumbnail"]
    x = feature["properties"]["origin_x"]
    y = feature["properties"]["origin_y"]
    resolution = feature["properties"]["pixel_resolution"]
    columns = feature["properties"]["columns"]

    img_path = path + id + ".png"
    # Get the thumbnail from the url by including the API key and the width
    img_url = thumb_url + "?api_key=5affbba496b5487b8ca1ba4e89a2cd8d&width=" + str(res)
    resp = urllib.request.urlopen(img_url)
    img = np.asarray(bytearray(resp.read()), dtype="uint8")
    img = cv2.imdecode(img, cv2.IMREAD_COLOR)
    os.chdir(path)
    sys.path.append(path + search_id + '/')
    cv2.imwrite(id+".png", img)
    png_imgs.append([img, img_path, id, x, y, resolution, columns])  
  
  return png_imgs


In [ ]:
# Make imgs transparent tifs
def save_transparent_tifs(input_path):
  cmds = []
  tif_imgs = []

  for img in os.listdir(input_path):
    if img[-3:] == "png":
      planet_id = img[:-4]
      planet_tif = "{}_transparent.tif".format(planet_id)
      cmds.append("convert -trim {}.png {}".format(planet_id, planet_tif))
      tif_imgs.append(planet_tif)

  for cmd in cmds:
      call(cmd.split(" "))

  return tif_imgs
  

In [ ]:
# Crop images to show only the current search area
def crop_img(geom, merged_img, img_path):
  geom_crop = [{'type': 'Polygon', 'coordinates': [[(geom['coordinates'][0][0][0], geom['coordinates'][0][0][1]),
                                                    (geom['coordinates'][0][1][0], geom['coordinates'][0][1][1]),
                                                    (geom['coordinates'][0][2][0], geom['coordinates'][0][2][1]),
                                                    (geom['coordinates'][0][3][0], geom['coordinates'][0][3][1]),
                                                    (geom['coordinates'][0][4][0], geom['coordinates'][0][4][1]),
    ]]}]

  if os.path.exists(merged_img):
    try:
        with rasterio.open(merged_img) as src:
          out_image, out_transform = mask(src, geom_crop, crop=True)
        out_meta = src.meta.copy()
        # save the resulting raster  
        out_meta.update({"driver": "GTiff",
            "height": out_image.shape[1],
            "width": out_image.shape[2],
        "transform": out_transform})

        # save the final image to google drive
        with rasterio.open(img_path + ".tif", "w", **out_meta) as dest:
            dest.write(out_image)
    except:
      print("Crop outide of returned images")
    


In [ ]:
# Return a list of dates to search
def get_dates(date_interval, start, end):
  dates = []
  date = start
  while date < end:
    dates.append(date)
    date = date + np.timedelta64(date_interval, 'M')
  return dates

In [ ]:
# Divide an area into multiple geometrys by size
def get_geoms(area, size, lat_steps, long_steps):
  all_coords = []
  start_lat, end_lat =  area[2][1], area[1][1]
  start_long, end_long = area[0][0], area[1][0]

  for lat in np.linspace(start_lat-size, end_lat, lat_steps):
    for long in np.linspace(start_long, end_long-size, long_steps):
      all_coords.append( { 
          "type": "Polygon",
          "coordinates": [
                    [
                      [long, lat],               # lower left
                      [long + size, lat],        # lower right
                      [long + size, lat + size],               # upper right
                      [long, lat + size],                      # upper left
                      [long, lat]                # lower left
          ]]})
  return all_coords

In [ ]:
# Perform a search to the API and save images
def search_planet(date, num_days, geom, geom_id, res):
  date_str = np.datetime_as_string(date)

  # Create filters for search
  # <10% cloud coverage
  cloud_cover_filter = {
    "type": "RangeFilter",
    "field_name": "cloud_cover",
    "config": {
      "lte": 0.5
    }
  }

  # Date filter from date and number of days we will search for
  lte_date = date + np.timedelta64(num_days, 'D')  # get end date by adding number of days specified
  gt_date_str = np.datetime_as_string(date) + 'T00:00:00Z' # format date string for Planet API
  lte_date_str = np.datetime_as_string(lte_date) + 'T00:00:00Z'

  date_filter = {
    "type": "DateRangeFilter",
    "field_name": "acquired",
    "config": {
      "gt": gt_date_str,
      "lte": lte_date_str
    }
  }

  # Geometry filter from specified coordinates
  geometry_filter = {
    "type": "GeometryFilter",
    "field_name": "geometry",
    "config": geom
  }

  # Combine filters
  combined_filter = {
    "type": "AndFilter",
    "config": [geometry_filter, cloud_cover_filter, date_filter]
  }

  # Specify item type 
  item_types = ["PSScene3Band"]

  # Create request from item type and  filter
  request = {
    "item_types" : item_types,
    "filter" : combined_filter
  }

  # Perform quick search to the API
  results=session.post(stats_url, json=request)
  quick_url = "{}/quick-search".format(URL)
  results = session.post(quick_url, json=request)
  geojson = results.json()
  features = geojson["features"] # Features is a list of dictionaries of all results 

  # Check if no images are found
  if len(features) != 0:
    # Images must have the same epsg code to be merged correctly
    # Create a list of all epsg codes and only use those with the most common code
    features_epsg = []
    epsg_codes = []
    for f in features:
      epsg_codes.append(f["properties"]["epsg_code"])
    
    epsg_codes = Counter(epsg_codes)
    epsg = epsg_codes.most_common(1)
    for f in features:
      if f["properties"]["epsg_code"] == epsg[0][0]:
        features_epsg.append(f)

    max_lat, min_lat, max_long, min_long = get_corners(features_epsg)
    
    path = "/content/{}/{}/".format(geom_id, date_str)
    search_id = geom_id + '_' + date_str

    # To merge the images into one covering the specified area we need to:
    # 1. Save the images as .png
    # 2. Use the png image and coordinates from the API results to save corresponding .wld (world files)
    # 3. Save the images as .tif and then convert them to geotiff using the .wld files
    # 4. Merge the images into one
    # 5. Crop the image to only show the search location

    # Save imgs as pngs
    png_imgs = save_pngs(features_epsg, path, search_id, res)
    # Save tif files
    tif_imgs = save_transparent_tifs(path)

    # add location data to transparent images
    imgs_to_merge = ""
    for feature in features_epsg:
      coords = feature['geometry']['coordinates'][0]
      ulx = min([coords[0][0], coords[1][0], coords[2][0], coords[3][0]])
      uly = max([coords[0][1], coords[1][1], coords[2][1], coords[3][1]])
      lrx = max([coords[0][0], coords[1][0], coords[2][0], coords[3][0]])
      lry = min([coords[0][1], coords[1][1], coords[2][1], coords[3][1]])
      current_tif = path + str(feature["id"]) + "_transparent.tif"
      new_tif = path + str(feature["id"]) + ".tif"
      #!gdal_translate -a_ullr $ulx $uly $lrx $lry $current_tif $new_tif
      cmd = "gdal_translate -a_ullr {} {} {} {} {} {}".format(ulx, uly, lrx, lry, current_tif, new_tif)
      call(cmd.split(" "))
      imgs_to_merge += new_tif + " "
    
    imgs_to_merge = imgs_to_merge[:-1]
    merged_img =  path + search_id + '.tif' # Set file location for the merged image
 
    # Merge images into one .tiff file
    !rio merge $imgs_to_merge --output $merged_img

    # Change the coordinates of the merged image
    !gdal_edit.py -a_ullr $min_long $max_lat $max_long $min_lat $merged_img
    
    # crop image to each adjacent area
    crop_img(geom, merged_img, path + search_id + "_cropped")

    # save cropped image 
    cropped_img = cv2.imread(path + search_id + "_cropped.tif")
    cv2.imwrite("/content/drive/My Drive/Colab Notebooks/FYP/Sequences1/" + search_id + ".tif", cropped_img)
        

In [ ]:
# Create sequence of images

# Set parameters
date_interval = 3   # Number of months between stage in sequence
start = np.datetime64('2019-01')  # Date to start sequence
end = np.datetime64('2022-04')    # Date to end sequence
num_days = 50        # Number of days to collect images
size = 1         # Size to divide each area by
res = 512        # Resolution of thumbnails images
classify_model = load_model("/content/drive/My Drive/Colab Notebooks/FYP/classification_model.h5", custom_objects={"f2_score": f2_score})

# Get list of dates to search
dates = get_dates(date_interval, start, end)
# Get locations
area = [
        [-68, -14], 
        [-50, -14],
        [-50, 0],
        [-68, 0],
        [-68, -14]
    ]

geoms = get_geoms(area, size, 14, 18)
print(len(geoms))
num_sequences = 100

# Iterate through locations, to create sequence of images
for g, geom in enumerate(geoms):
  print("Searching location: {}/{}".format(g, num_sequences))
  geom_images = np.zeros((13, 400, 64, 64, 3))
  if not os.path.exists('/content/' + str(g) + '/'):
    os.mkdir('/content/' + str(g) + '/') # Create directory for geom
  for d, date in enumerate(dates):
    date = date + np.timedelta64(0, 'D')
    str_date = np.datetime_as_string(date)
    print("Searching date: " + str_date)
    if not os.path.exists('/content/{}/{}/'.format(str(g), str_date)):
      os.mkdir('/content/{}/{}/'.format(str(g), str_date))  # Create directory for date
    if not os.path.exists("/content/{}/{}/{}_{}_cropped.tif".format(g, str_date, g, str_date)):
      search_planet(date, num_days, geom, str(g), res)
    search_id = str(g) + "_" + str_date
    cropped_img = "/content/{}/{}/{}_cropped.tif".format(g, str_date, search_id)
    current_area = geom['coordinates'][0]
    cropped_img = cv2.imread(cropped_img)
    ulx, uly, lrx, lry = current_area[3][0], current_area[3][1], current_area[1][0], current_area[1][1]
    !gdal_edit.py -a_ullr $ulx $uly $lrx $lry $cropped_img
    # crop images and make predictions to build map layer
    sub_path = '/content/{}/{}/sub_geoms/'.format(str(g), str_date)
    if not os.path.exists(sub_path):
      os.mkdir(sub_path)
    sub_geoms = get_geoms(current_area, 0.05, 20, 20)
    for sg, sub_geom in enumerate(sub_geoms):
      img_path = sub_path + search_id + "_" + str(sg)
      if not os.path.exists(img_path + ".tif"):
        crop_img(sub_geom, cropped_img_path2, img_path)
        img = cv2.imread(img_path + ".tif")
        img = cv2.resize(img, (64, 64))
        geom_images[d, sg] = img
  np.save("/content/drive/My Drive/Colab Notebooks/FYP/sequence training data/" + str(g), geom_images)